In [ ]:
pip install transformers[torch] datasets

In [ ]:
from datasets import load_dataset
import numpy as np

In [ ]:
raw_datasets = load_dataset("glue", "sst2")
dir(raw_datasets['train'])

### Inspecting the dataset

In [ ]:
type(raw_datasets['train'])

In [ ]:
raw_datasets['train'].data

In [ ]:
raw_datasets['train'][50000:50003]

In [ ]:
raw_datasets['train'].features

In [ ]:
from transformers import AutoTokenizer

### Setting up the Tokenizer

In [ ]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
tokenized_sentences = tokenizer(raw_datasets['train'][0:3]['sentence'])
from pprint import pprint
pprint(tokenized_sentences)

In [ ]:
def tokenize_fn(batch):
  return tokenizer(batch['sentence'], truncation=True)

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_fn, batched=True)

In [ ]:
from transformers import TrainingArguments

In [ ]:
training_args = TrainingArguments(
    "my_trainer",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=1
)

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels=2
)

In [ ]:
type(model)

In [ ]:
model

In [ ]:
!pip install torchinfo

In [ ]:
from torchinfo import summary

summary(model)

### Used in order to check that the fine tuning actually changed something in the model

In [ ]:
params_before = []
for name, p in model.named_parameters():
  params_before.append(p.detach().cpu().numpy())

In [ ]:
from transformers import Trainer

In [ ]:
from datasets import load_metric

In [ ]:
metric = load_metric("glue", "sst2")

In [ ]:
metric.compute(predictions=[1, 0, 1], references=[1, 0, 0])

In [ ]:
def compute_metrics(logits_and_labels):
  logits, labels = logits_and_labels
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model('my_saved_model')

In [ ]:
!ls my_saved_model

In [ ]:
from transformers import pipeline
newmodel = pipeline('text-classification', model='my_saved_model', device=0)

In [ ]:
newmodel('This movie is great!')

In [ ]:
newmodel('This movie sucks')

In [ ]:
!cat my_saved_model/config.json

In [ ]:
import json

### Modifying the labels so they display properly

In [ ]:
config_path = "my_saved_model/config.json"
with open(config_path) as f:
  j = json.load(f)

j['id2label'] = {0: 'negative', 1: 'positive'}

with open(config_path, 'w') as f:
  json.dump(j, f, indent=2)

In [ ]:
!cat my_saved_model/config.json

In [ ]:
newmodel = pipeline('text-classification', model='my_saved_model', device=0)

In [ ]:
newmodel('This movie is great!')

In [ ]:
params_after = []
for name, p in model.named_parameters():
  params_after.append(p.detach().cpu().numpy())

In [ ]:
for p1, p2 in zip(params_before, params_after):
  print(np.sum(np.abs(p1 - p2)))